# Exploring and fixing data with Synapse Spark

In this task, you will use a Synapse Spark notebook to explore a few of the files in the `wwi-02/sale-poc` folder in the data lake. You will also use Python code to fix the issues with the `sale-20170502.csv` file, so all the files in the directory can be ingested using a Synapse Pipeline later in this lab.

The first thing we need to is set a variable within the notebook to provide the name of your primary data lake storage account. Before executing the cell below, you will need to replace `[YOUR-DATA-LAKE-ACCOUNT-NAME]` with the name of the primary data lake storage account associated with your Syanpse workspace.

You can find the name of your data lake storage account by navigating to the **Data** hub in Synapse Studio, selecting the **Linked** tab, and locating the storage account name that begins with **asadatalake** under **Azure Data Lake Storage Gen2**.

![The primary data lake storage account is highlighted on the Data hub's Linked tab.](https://solliancepublicdata.blob.core.windows.net/images/synapse/data-hub-primary-data-lake-storage-account.png "Primary ADLS Gen2 Account")

1. Copy the name of your data lake storage account and paste it in place of `[YOUR-DATA-LAKE-ACCOUNT-NAME]` in the cell below, then execute the cell by selecting the **Run cell** button that becomes visible when you select the cell.

    ![The primary data lake storage account is highlighted on the Data hub's Linked tab.](https://solliancepublicdata.blob.core.windows.net/images/synapse/synapse-notebook-run-cell.png "Primary ADLS Gen2 Account")



In [ ]:
adls_account_name = '[YOUR-DATA-LAKE-ACCOUNT-NAME]'

## Exploring files with Spark

1. The first step in exploring data using Synapse Spark is to load a file from the data lake. For this, we can use the `spark.read.load()` method of the `SparkSession`.

2. In Spark, we can load the data from our files into [DataFrames](https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#datasets-and-dataframes), which are an abstraction that allows data to be structured in named columns. Execute the cell below to load the data from the `sale-20170501.csv` file into a data frame. You can run the cell by hovering your mouse over the left-hand side of the cell and then selecting the blue **Run cell** button.

    ![The Run Cell button is highlighted to the left of the cell contents to execute.](https://solliancepublicdata.blob.core.windows.net/images/synapse/synapse-notebook-run-cell-load-sale-20170501-csv.png "Run cell")

In [ ]:
# First, load the file `sale-20170501.csv` file, which we know from our previous exploration to be formatted correctly.
# Note the use of the `header` and `inferSchema` parameters. Header indicates the first row of the file contains column headers,
# and `inferSchema` instruct Spark to use data within the file to infer data types.
df = spark.read.load(f'abfss://wwi-02@{adls_account_name}.dfs.core.windows.net/sale-poc/sale-20170501.csv', format='csv', header=True, inferSchema=True)

## View the contents of the DataFrame

With the data from the `sale-20170501.csv` file loaded into a data frame, we can now use various methods of a data frame to explore the properties of the data.

1. First, let's look at the data as it was imported. Execute the cell below to view and inspect the data in the data frame.

In [ ]:
display(df.limit(10))

2. Like we saw duing exploration with the SQL on-demand capabilities of Azure Synapse, Spark allows us to view and query against the data contained within files. 

3. Now, use the `printSchema()` method of the data frame to view the results of using the `inferSchema` parameter when creating the data frame. Execute the cell below and observe the output.

In [ ]:
# Now, print the inferred schema. We will need this information below to help with the missing headers in the May 2, 2017 file.
df.printSchema()

4. The `printSchema` method outputs both field names and data types that are based on the Spark engine's evaluation of the data contained within each field.

    > We can use this information later to help define the schema for the poorly formed `sale-20170502.csv` file. In addition to the field names and data types, we should note the number of features or columns contained in the file. In this case, note that there are 11 fields. That will be used to determine where to split the single row of data.

5. As an example of further exploration we can do, run the cell below to create and display a new data frame that contains an ordered list of distinct Customer and Product Id pairings. We can use these types of functions to find invalid or empty values quickly in targeted fields.

In [ ]:
# Create a new data frame containing a list of distinct CustomerId and ProductId values in descending order of the CustomerId.
df_distinct_products = df.select('CustomerId', 'ProductId').distinct().orderBy('CustomerId')

# Display the first 100 rows of the resulting data frame.
display(df_distinct_products.limit(100))

6. Next, let's attempt to open and explore the `sale-20170502.csv` file using the `load()` method, as we did above.

In [ ]:
# Next, let's try to read in the May 2, 2017 file using the same `load()` method we used for the first file.
df = spark.read.load(f'abfss://wwi-02@{adls_account_name}.dfs.core.windows.net/sale-poc/sale-20170502.csv', format='csv')
display(df.limit(10))

7. As we saw in T-SQL, we receive a similar error in Spark that the number of columns processed may have exceeded limit of 20480 columns. To work with the data in this file, we need to use more advanced methods, as you will see in the next section below.


## Handling and fixing poorly formed CSV files

> The steps below provide example code for fixing the poorly-formed CSV file, `sale-20170502.csv` we discovered during exploration of the files in the `wwi-02/sale-poc` folder. This is just one of many ways to handle "fixing" a poorly-formed CSV file using Spark.

1. To "fix" the bad file, we need to take a programmatic approach, using Python to read in the contents of the file and then parse them to put them into the proper shape.

    > To handle the data being in a single row, we can use the `textFile()` method of our `SparkContext` to read the file as a collection of rows into a resilient distributed dataset (RDD). This allows us to get around the errors around the number of columns because we are essentially getting a single string value stored in a single column.

2. Execute the cell below to load the RDD with data from the file.

In [ ]:
# Import the NumPy library. NumPy is a python library used for working with arrays.
import numpy as np

# Read the CSV file into a resilient distributed dataset (RDD) as a text file. This will read each row of the file into rows in an RDD.
rdd = sc.textFile(f'abfss://wwi-02@{adls_account_name}.dfs.core.windows.net/sale-poc/sale-20170502.csv')

3. With the data now stored in an RDD, we can access the first, and only, populated row in the RDD, and split that into individual fields. We know from our inspection of the file in Notepad++ that it all the fields are separated by a comma (,), so let's start by splitting on that to create an array of field values. Execute the cell below to create a data array.

In [ ]:
# Since we know there is only one row, grab the first row of the RDD and split in on the field delimiter (comma).
data = rdd.first().split(',')

field_count = len(data)
# Print out the count of fields read into the array.
print(field_count)

4. By splitting the row on the field delimiter, we created an array of all the individual field values in the file, the count of which you can see above.

5. Now, run the cell below to do a quick calculation on the expected number of rows that will be generated by parsing every 11 fields into a single row.

In [ ]:
import math

expected_row_count = math.floor(field_count / 11)
print(f'The expected row count is: {expected_row_count}')

6. Next, let's create an array to store the data associated with each "row".

    > We will set the max_index to the number of columns that are expected in each row. We know from our exploration of other files in the `wwi-02/sale-poc` folder that they contain 11 columns, so that is the value we will set.

7. In addition to setting variables, we will use the cell below to loop through the `data` array and assign every 11 values to a row. By doing this, we are able to "split" the data that was once a single row into appropriate rows containing the proper data and columns from the file.

8. Execute the cell below to create an array of rows from the file data.

In [ ]:
# Create an array to store the data associated with each "row". Set the max_index to the number of columns that are in each row. This is 11, which we noted above when viewing the schema of the May 1 file.
row_list = []
max_index = 11

# Now, we are going to loop through the array of values extracted from the single row of the file and build rows consisting of 11 columns.
while max_index <= len(data):
    row = [data[i] for i in np.arange(max_index-11, max_index)]
    row_list.append(row)

    max_index += 11

print(f'The row array contains {len(row_list)} rows. The expected number of rows was {expected_row_count}.')

9. The last thing we need to do to be able to work with the file data as rows is to read it into a Spark DataFrame. In the cell below, we use the `createDataFrame()` method to convert the `row_list` array into a data frame, which also adding names for the columns. Column names are based on the schema we observed in the well formatted files in the `wwi-02/sale-poc` directory.

10. Execute the cell below to create a data frame containing row data from the file and then display the first 10 rows.

In [ ]:
# Finally, we can use the row_list we created above to create a DataFrame. We can add to this a schema parameter, which contains the column names we saw in the schema of the first file.
df_fixed = spark.createDataFrame(row_list,schema=['TransactionId', 'CustomerId', 'ProductId', 'Quantity', 'Price', 'TotalAmount', 'TransactionDateId', 'ProfitAmount', 'Hour', 'Minute', 'StoreId'])
display(df_fixed.limit(10))

## Write the "fixed" file into the data lake

1. The last step we will take as part of our exploration and file fixing process is to write the data back into the data lake, so it can be ingested following the same process as the other files in the `wwi-02/sale-poc` folder.

2. Execute the cell below to save the data frame into the data lake a series of files in a folder named `sale-20170502-fixed`.

    > Note: Spark parallelizes workloads across worker nodes, so when saving files, you will notice they are saved as a collection "part" files, and not as a single file. While there are some libraries you can use to create a single file, it is helpful to get used to working with files generated via Spark notebooks as they are natively created.


In [ ]:
df.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save(f'abfss://wwi-02@{adls_account_name}.dfs.core.windows.net/sale-poc/sale-20170502-fixed')

## Inspect the fixed file in the data lake

1. With the fixed file written to the data lake, you can quickly inpsect it to verify the files are now formatted properly. Select the `wwi-02` tab above and then double-click on the `sale-20170502-fixed` folder.

    ![](https://solliancepublicdata.blob.core.windows.net/images/synapse/wwi-02-sale-poc-sale-20170502-fixed.png)

2. In the `sale-20170502-fixed` folder, right-click the first file whose name begins with `part` and whose extension is `.csv` and select **Preview** from the context menu.

    ![](https://solliancepublicdata.blob.core.windows.net/images/synapse/wwi-02-sale-poc-sale-20170502-fixed-content.png)

3. In the **Preview** dialog, verify you see the proper columns and that the data looks valid in each field.

    ![](https://solliancepublicdata.blob.core.windows.net/images/synapse/sale-20170502-fixed-preview.png)

## Wrap-up

Throughout this exercise, you used a Spark notebook to explore data stored within files in the data lake. You used Python code to extract data from a poorly formatted CSV file, assemble the data from that file into proper rows, and then write the "fixed" file back out into your data lake.

You can now return to the lab guide to continue with the next section of Lab 2.
